<a href="https://colab.research.google.com/github/sezinbiner/Covid_X_ray_Classification/blob/main/unbalanced_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)
generator_dir = "/content/gdrive/MyDrive/Kaggle/COVID-19_Radiography_Dataset"

class_names = ['COVID', 'Normal']
generator = datagen.flow_from_directory(
        generator_dir,
        target_size=(224, 224),  
        batch_size=20,
        shuffle=True,
        class_mode='binary')


Found 13808 images belonging to 2 classes.


In [3]:
def extract_features(number_of_images, generator, batch_size):
  features = np.zeros(shape=(number_of_images, extraction_layer.output_shape[1], extraction_layer.output_shape[2], extraction_layer.output_shape[3]))
  labels = np.zeros(shape=(number_of_images))
  i = 0
  print("loop")
  for inputs_batch, labels_batch in generator:
      #print(i)
      features_batch = extraction_layer.predict(inputs_batch)
      features[i * batch_size : (i + 1) * batch_size] = features_batch
      labels[i * batch_size : (i + 1) * batch_size] = labels_batch
      i += 1
      if i * batch_size >= number_of_images:
          break
  print("loop over")
  return features, labels

In [ ]:
from tensorflow.python.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.python.keras.applications.densenet import DenseNet201
from tensorflow.python.keras.applications.inception_v3 import InceptionV3
from tensorflow.python.keras.applications.nasnet import NASNetMobile
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.applications.vgg19 import VGG19
import numpy as np
from keras.models import Model

extraction = ResNet50V2(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)
extraction.summary()

In [5]:
extraction_layer = Model(extraction.input, extraction.layers[-5].output)
print(extraction_layer.output_shape)

(None, 7, 7, 512)


In [ ]:
features,labels = extract_features(13808, generator, generator.batch_size)

loop


In [ ]:
from keras.models import Sequential
from keras import models
from keras import layers
model = models.Sequential()
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(
    loss='binary_crossentropy',
    optimizer= "adam" ,
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
from sklearn.model_selection import KFold                    
kf = KFold(n_splits=5)
X = features
y = labels

In [ ]:
index_list = []
test_index_list = []
#Trying different train and test sets for higher accuracy
for train_index,test_index in kf.split(X):
    index_list.append(train_index)
    index_list.append(test_index)

In [ ]:
X_train = X[index_list[0]]                           
X_test = X[index_list[1]]
y_train = y[index_list[0]]
y_test = y[index_list[1]]
print("Train:",index_list[0], "Length:", len(y_train),  "Test:", index_list[1], "Length:", len(y_test))

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=20,
                    validation_split=0.2)
print("\n")
from sklearn import metrics
y_pred = model.predict(X_test)
probas = np.array(y_pred)
labels = []
for prob in probas:
  if prob < 0.5:
    labels.append(0)
  else:
    labels.append(1)
y_true_labels = y_test
y_pred_labels = labels
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_true_labels, y_pred_labels)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_true_labels, y_pred_labels,pos_label='positive', average='micro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_true_labels, y_pred_labels, average='micro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_true_labels, y_pred_labels, average='weighted')
print('F1 score: %f' % f1)
from sklearn.metrics import classification_report
print(classification_report(y_true_labels, y_pred_labels, target_names=["covid", "normal"]))
from sklearn.metrics import confusion_matrix
import seaborn as sn
df_cm = pd.DataFrame(confusion_matrix(y_true_labels, y_pred_labels), 2, 2)
print()
sn.heatmap(df_cm, annot=True)

In [ ]:
model.save("/content/gdrive/MyDrive/model_k_fold/nasnet_fold2.h5")